# extracting stimuli from word file

In [262]:
import docx
from docx import Document
import json

document = Document('stimuli_s1e3.docx')

doc_dict = {}
filler_status = None

for par in document.paragraphs:
    if par.style.name == 'Heading':
        if 'филлер' in par.text:
            filler_status = par.text
            doc_dict[filler_status] = []
        else:
            colloc_status = par.text
            if colloc_status not in doc_dict.keys():
                doc_dict[colloc_status] = dict()
    elif par.style.name == 'Heading 2':
        mod_status = par.text
        if mod_status not in doc_dict[colloc_status].keys():
            doc_dict[colloc_status][mod_status] = dict()
    elif par.style.name == 'Heading 3':
        case_status = par.text
        if case_status not in doc_dict[colloc_status][mod_status].keys():
            doc_dict[colloc_status][mod_status][case_status] = []
    else:
        if filler_status:
            doc_dict[filler_status].append(par.text)
        else:
            doc_dict[colloc_status][mod_status][case_status].append(par.text)
            
with open('doc_dict.json', 'w') as f:
    json.dump(doc_dict, f)

In [263]:
import pandas as pd

df = pd.json_normalize(doc_dict, max_level=3)
df[df.columns[2:]]

,С коллокациями.Сам.Датив,С коллокациями.Сам.Номинатив,С коллокациями.Один.Датив,С коллокациями.Один.Номинатив,С коллокациями.Adj.Инструменталис,С коллокациями.Adj.Номинатив,С неколлокациями.Сам.Датив,С неколлокациями.Сам.Номинатив,С неколлокациями.Один.Датив,С неколлокациями.Один.Номинатив,С неколлокациями.Adj.Инструменталис,С неколлокациями.Adj.Номинатив
0,"[Соседи предлагали поухаживать за кроликом, по...","[В коробке с конструктором была инструкция, но...",[Николай Андреевич принял решение поехать в от...,[Лиза приняла решение выступить на конференции...,[Андрей принял решение начать учебный год трез...,"[Несмотря на плохое самочувствие, Наташа приня...",[Гриша как следует обдумал решение начать зара...,[Гриша как следует обдумал решение начать зара...,[Витя долго не раскрывал решение уехать в Дуба...,[Даша объяснила решение делать поделку одна те...,[Вчера на речке к вожатой подошел мальчик из т...,[После того как Нина очнулась после обморока в...


In [264]:
# gram fillers
Fg = pd.DataFrame(columns=['ITEM','CONDITION','MODIFIER','CASE','SENTENCE','TYPE'])
Fg['SENTENCE'] = doc_dict['Грамматичные филлеры']
Fg['TYPE'] = 'Fg'

#ungram fillers
Fu = pd.DataFrame(columns=['ITEM','CONDITION','MODIFIER','CASE','SENTENCE','TYPE'])
Fu['SENTENCE'] = doc_dict['Неграмматичные филлеры']
Fu['TYPE'] = 'Fu'

In [265]:
from sklearn import preprocessing

columns = ['ITEM','CONDITION','MODIFIER','CASE','SENTENCE','TYPE']

items = pd.DataFrame(columns=columns)
items['TYPE'] = 'item'

for col in df.columns[2:]:
    items_list = df[col].values[0]
    items_df = pd.DataFrame(columns=columns)
    items_df['SENTENCE'] = df[col]
    CONDITION, MODIFIER, CASE = col.split('.')
    items_df['CONDITION'] = CONDITION
    items_df['MODIFIER'] = MODIFIER
    items_df['CASE'] = CASE
    items_df['TYPE'] = 'item'
    items = items.append(items_df)
    
items['CONDITION'] = items['CONDITION'].map({
    'С коллокациями': 'colloc',
    'С неколлокациями': 'no_colloc'
})

items['MODIFIER'] = items['MODIFIER'].map({
    'Сам': 'sam',
    'Один': 'odin',
    'Adj': 'adj'
})

items['CASE'] = items['CASE'].map({
    'Датив': 'non_nom',
    'Номинатив': 'nom',
    'Инструменталис': 'non_nom'
})

items = items.explode('SENTENCE')
items = items.append(Fg).append(Fu)
items = items.fillna(value = '')
items['FILLER_TYPE'] = items['TYPE']
items['TYPE'] = items['TYPE'].map({'Fg':'filler', 'Fu':'filler', 'item':'item'})
items['cond'] = items['CONDITION'].values + items['MODIFIER'].values + items['CASE'].values + items['FILLER_TYPE'].values
items = items.reset_index().drop(columns=['index', 'ITEM']).reset_index().rename(columns={
    'index':'ITEM', 'CONDITION':'COLLOCATION'})

le = preprocessing.LabelEncoder()
items['CONDITION'] = le.fit_transform(items['cond'])
# items['LIST'] = items['CONDITION']
items.drop(columns=['cond'], inplace=True)

items

,ITEM,COLLOCATION,MODIFIER,CASE,SENTENCE,TYPE,FILLER_TYPE,CONDITION
0,0,colloc,sam,non_nom,"Соседи предлагали поухаживать за кроликом, пок...",item,item,7
1,1,colloc,sam,non_nom,Не надо переубеждать Катю — она имеет право са...,item,item,7
2,2,colloc,sam,non_nom,Девочки из второго отряда получили разрешение ...,item,item,7
3,3,colloc,sam,non_nom,"После того, как Анжела сходила с нами на конюш...",item,item,7
4,4,colloc,sam,non_nom,Антон хотел послать в Нижний Новгород своего п...,item,item,7
...,...,...,...,...,...,...,...,...
103,103,,,,"Арина вышила крестиком платочком, чтобы подари...",filler,Fu,1
104,104,,,,Никита закончил университету в этом году и теп...,filler,Fu,1
105,105,,,,"Алиса обычно стесняется петь при гостей, это о...",filler,Fu,1
106,106,,,,Детям в начальной школе нужно позволяют писать...,filler,Fu,1


In [266]:
items['FILLER_TYPE'].value_counts()

item    72
Fu      18
Fg      18
Name: FILLER_TYPE, dtype: int64

In [267]:
items.to_csv('experiment.csv', index=False)

## making lists

In [268]:
def make_list(df, idx):
    exp_list = pd.DataFrame(columns=df.columns)
    i = idx
    for cond in df['CONDITION'].unique():
        cond_df = df.loc[df['CONDITION'] == cond]
        if 'item' in cond_df['FILLER_TYPE'].tolist():
            exp_list = exp_list.append(cond_df.reset_index().loc[i])
        elif 'Fu' in cond_df['FILLER_TYPE'].tolist():
            exp_list = exp_list.append(cond_df.sample(12))
        elif 'Fg' in cond_df['FILLER_TYPE'].tolist():
            exp_list = exp_list.append(cond_df.sample(12))
        i = (i + 1) % 6
    exp_list['LIST'] = int(idx)
    
    return exp_list.drop(columns=['index'])

In [269]:
make_list(items, 0)

,ITEM,COLLOCATION,MODIFIER,CASE,SENTENCE,TYPE,FILLER_TYPE,CONDITION,LIST
0,0,colloc,sam,non_nom,"Соседи предлагали поухаживать за кроликом, пок...",item,item,7,0
1,7,colloc,sam,nom,Сотрудники отдела IT-инфраструктуры имеют прав...,item,item,6,0
2,14,colloc,odin,non_nom,Моя бабушка постоянно за меня волнуется — вчер...,item,item,5,0
3,21,colloc,odin,nom,"Все дети в классе выбрали себе напарника, и то...",item,item,4,0
4,28,colloc,adj,non_nom,"Миша очень волновался, но дал согласие выступи...",item,item,3,0
5,35,colloc,adj,nom,Лёша был самым привередливым мальчиком в отряд...,item,item,2,0
0,36,no_colloc,sam,non_nom,Гриша как следует обдумал решение начать зараб...,item,item,13,0
1,43,no_colloc,sam,nom,Костя упорно отстаивает право сам готовить себ...,item,item,12,0
2,50,no_colloc,odin,non_nom,Женя жил со своим братом в одной комнате много...,item,item,11,0
3,57,no_colloc,odin,nom,Назойливый ухажер уже полчаса мотал Фиме нервы...,item,item,10,0


In [270]:
exp_df = pd.DataFrame(columns=items.columns)

for idx in range(6):
    exp_df = exp_df.append(make_list(items, idx))
    
exp_df.to_csv('experiment.csv')

In [224]:
exp_df

,ITEM,COLLOCATION,MODIFIER,CASE,SENTENCE,TYPE,CONDITION,FILLER_TYPE,LIST
0,0,colloc,sam,non_nom,"Соседи предлагали поухаживать за кроликом, пок...",item,5,item,0.0
0,6,colloc,sam,nom,"В коробке с конструктором была инструкция, но ...",item,4,item,0.0
0,12,colloc,odin,non_nom,Николай Андреевич принял решение поехать в отп...,item,3,item,0.0
0,18,colloc,odin,nom,Лиза приняла решение выступить на конференции ...,item,2,item,0.0
0,24,colloc,adj,non_nom,Андрей принял решение начать учебный год трезвым.,item,1,item,0.0
...,...,...,...,...,...,...,...,...,...
97,97,NaN,NaN,NaN,"Лиза сегодня не сможет погулять с Рексом, ей н...",filler,12,Fg,5.0
100,100,NaN,NaN,NaN,Сколько Валя ни пыталась взбить сливки венчико...,filler,12,Fg,5.0
87,87,NaN,NaN,NaN,На станции Мнёвники в конце 2021 года был зафи...,filler,12,Fg,5.0
78,78,NaN,NaN,NaN,"Вместо того, чтобы каждый раз брать целлофанов...",filler,12,Fg,5.0


In [201]:
for col in items.columns:
    print(col, '\n', items[col].value_counts())

ITEM 
 0      1
68     1
79     1
78     1
77     1
      ..
32     1
31     1
30     1
29     1
107    1
Name: ITEM, Length: 108, dtype: int64
COLLOCATION 
 colloc       36
no_colloc    36
Name: COLLOCATION, dtype: int64
MODIFIER 
 odin    24
sam     24
adj     24
Name: MODIFIER, dtype: int64
CASE 
 non_nom    36
nom        36
Name: CASE, dtype: int64
SENTENCE 
 Как только самолет оторвался от земли, Анечка начала трястись от страха. Она никогда не летала на самолете и к тому же боялась высоты.                                           2
На станции Мнёвники в конце 2021 года был зафиксирован самый низкий пассажиропоток в истории метро: на ней побывало всего 11 человек за день.                                    2
Мы были поражены тем, насколько четко видны звезды из ночном небе над Байкалом: городское световое загрязнение всегда прячет такую красоту.                                      2
На олимпиаду от математического кружка поехало 9 детей — почти вдвое больше, чем в прошлом году. 

## training stimuli

In [123]:
pd.DataFrame({
    'ITEM': [2001, 2002, 2003],
    'CONDITION': ['training', 'training', 'training'],
    'SENTENCE': [
        'Нами выходить через полчаса, ты ещё а не проснулись!',
        'Лиза обожает рисовать фломастерами, а когда они подсыхают, она смачивает их одеколоном.',
        'Доктора я помню, что не видел, когда он приехал.'
    ]
}).to_csv('exercise.csv', index=False)